# INFO-H420 Management of Data Science and Business Workflows

## Project on Responsible Data Science

## 3.Private-Fair Classifer

In [1]:
import pandas as pd
import numpy as np
from aif360.datasets import AdultDataset
from aif360.metrics import BinaryLabelDatasetMetric, ClassificationMetric
from aif360.algorithms.preprocessing import Reweighing
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report

from IPython.display import Markdown, display
import math
import random

c:\Users\theod\anaconda3\envs\omnixai\Lib\site-packages\inFairness\utils\ndcg.py:37: FutureWarning: We've integrated functorch into PyTorch. As the final step of the integration, `functorch.vmap` is deprecated as of PyTorch 2.0 and will be deleted in a future version of PyTorch >= 2.3. Please use `torch.vmap` instead; see the PyTorch 2.0 release notes and/or the `torch.func` migration guide for more details https://pytorch.org/docs/main/func.migrating.html
  vect_normalized_discounted_cumulative_gain = vmap(
c:\Users\theod\anaconda3\envs\omnixai\Lib\site-packages\inFairness\utils\ndcg.py:48: FutureWarning: We've integrated functorch into PyTorch. As the final step of the integration, `functorch.vmap` is deprecated as of PyTorch 2.0 and will be deleted in a future version of PyTorch >= 2.3. Please use `torch.vmap` instead; see the PyTorch 2.0 release notes and/or the `torch.func` migration guide for more details https://pytorch.org/docs/main/func.migrating.html
  monte_carlo_vect_ndcg =

### Load and preprocess the dataset

In [ ]:
def custom_preprocessing(df):
    median_age = df['age'].median()
    df['age_binary'] = df['age'].apply(lambda x: 0 if x <= median_age else 1)
    df = df.drop('age', axis=1)
    df['race'] = df['race'].apply(lambda x: 1 if x =="White"  else 0)
    df['sex'] =df['sex'].apply(lambda x: 1 if x =="Male"  else 0)
    return df
# So what we did is to add a new column 'age_binary' to the dataset and drop the 'age' column, in order to 
# binarise the age column.
# Load the dataset with the library aif360
dataset= AdultDataset(custom_preprocessing=custom_preprocessing,
                          protected_attribute_names=['age_binary', 'sex'], # race will remain because in the original library is defined with this protecte attribute
                          privileged_classes=[np.array([1.0]),np.array([1.0]) ]) # We supposed that the privileged class is the old white male. It's also defined like this in the original library

In [3]:
# Define the privileged and unprivileged groups in order to compute the disparate impact
privileged_groups = [{'age_binary': 1, 'sex': 1}]  # Old males
unprivileged_groups = [{'age_binary': 0, 'sex': 0}]  # Young females


### Split and train metric

In [4]:
# Get the dataset and split into train and test
dataset_orig_train, dataset_orig_vt = dataset.split([0.7], shuffle=True)
dataset_orig_valid, dataset_orig_test = dataset_orig_vt.split([0.5], shuffle=True)

In [5]:
# Compute the fairness metric statistical parity measure, which is the difference in the mean prediction between the unprivileged and privileged groups.
# A negative value indicates less favorable outcomes for the unprivileged groups. in order to see if the dataset is biased
metric_orig_train = BinaryLabelDatasetMetric(dataset_orig_train, 
                                             unprivileged_groups=unprivileged_groups, 
                                             privileged_groups=privileged_groups)
display(Markdown("#### Original training dataset"))
print("Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_orig_train.mean_difference())

#### Original training dataset

Difference in mean outcomes between unprivileged and privileged groups = -0.364527


### Compute fairness metric on transformed dataset


In [6]:
RW = Reweighing(unprivileged_groups=unprivileged_groups,
               privileged_groups=privileged_groups)
RW.fit(dataset_orig_train)
dataset_transf_train = RW.transform(dataset_orig_train)

In [7]:
metric_transf_train = BinaryLabelDatasetMetric(dataset_transf_train, 
                                               unprivileged_groups=unprivileged_groups,
                                               privileged_groups=privileged_groups)
display(Markdown("#### Transformed training dataset"))
print("Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_transf_train.mean_difference())

#### Transformed training dataset

Difference in mean outcomes between unprivileged and privileged groups = -0.000000


### Convert Fair_dataSet to DataFrame


In [ ]:
# Convert the datasets to pandas DataFrame
df = dataset_transf_train.convert_to_dataframe()[0]

### Privatice sensitive data 

In [10]:
def get_epsilon(p=0.75, q=0.75):
    return math.log( max(q/(1-p), p/(1-q)) )

def rand_resp(x, p=0.75, q=0.75):
    toss = random.random()
    if x == 0:
        y = 0 if toss <= q else 1
    else:
        y = 1 if toss <= p else 0
    return y

In [11]:
# Create a copy of the dataset
df_private = df.copy()
epsilon = get_epsilon()

# Appliquer LDP aux colonnes sensibles
df_private['age_binary_private'] = df['age_binary'].apply(lambda x: rand_resp(x))
df_private['sex_private'] = df['sex'].apply(lambda x: rand_resp(x))

### Split and Train private Data

In [21]:
# Divide into features (X) and labels (y)
X = df_private.drop('income-per-year', axis=1)
y = df_private['income-per-year']

# Divide into training and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Train an XGBoost classifier
model = XGBClassifier(eval_metric='logloss', random_state=42)
model.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='logloss',
              feature_types=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=None, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=None,
              n_jobs=None, num_parallel_tree=None, random_state=42, ...)

### Test the Model

In [22]:
# Predictions
y_pred = model.predict(X_test)

# Evaluate performance
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

Accuracy: 0.8714330841318311
Classification Report:
               precision    recall  f1-score   support

         0.0       0.90      0.94      0.92      7145
         1.0       0.78      0.67      0.72      2352

    accuracy                           0.87      9497
   macro avg       0.84      0.80      0.82      9497
weighted avg       0.87      0.87      0.87      9497

